In [1]:
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model('asl_sign_language_model.h5')  
labels = ['A', 'B', 'C', 'D', 'del', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'nothing', 'O', 'P', 'Q', 'R', 'S', 'space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [2]:
import cv2
import numpy as np

# Parameters
IMG_SIZE = 64  # Image size for model input
x, y, w, h = 300, 100, 300, 300  # Region of Interest (ROI) for hand signs

# Function to preprocess the image before passing it to the model
def preprocess_image(image):
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))  # Resize to model input size
    image = image.astype('float32') / 255.0  # Normalize pixel values
    return np.expand_dims(image, axis=0)  # Add batch dimension

# Start webcam and prediction loop
cap = cv2.VideoCapture(0)  # Open the webcam

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    roi = frame[y:y+h, x:x+w]  # Extract the region of interest (hand sign)

    # Process the image for model prediction
    processed = preprocess_image(roi)
    prediction = model.predict(processed, verbose=0)
    pred_index = np.argmax(prediction)  # Get the predicted class index
    pred_letter = labels[pred_index]  # Get the label for the predicted class

    # Draw a rectangle around the hand and display the prediction
    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    cv2.putText(frame, f"Prediction: {pred_letter}", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

    # Display the video with the predicted label
    cv2.imshow("ASL Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

cap.release()  # Release the webcam
cv2.destroyAllWindows()  # Close any open OpenCV windows


In [4]:
predictions = []
for _ in range(5):  # Collect 5 frames before predicting
    prediction = model.predict(processed, verbose=0)
    predictions.append(np.argmax(prediction))

final_prediction = np.bincount(predictions).argmax()  # Majority voting


In [3]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('asl_model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmpysi32s4l\assets


INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmpysi32s4l\assets


Saved artifact at 'C:\Users\ADMIN\AppData\Local\Temp\tmpysi32s4l'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 29), dtype=tf.float32, name=None)
Captures:
  1679358769872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1679358770640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1679358770832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1679360722512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1679360724240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1679360724816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1679360725008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1679360725968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1679360723472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1679360727120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  16793